In [ ]:
from pathlib import Path

import pandas as pd
import yaml
from utils import (
    amputees,
    intact,
    participants,
)

In [ ]:
paper_plots_path = Path("/Users/jorisg/Desktop/upper_limb/paper_figures_revision")
base_dir = Path().resolve().parent

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

x = np.linspace(-10, 10, 500)
softplus = np.log1p(np.exp(x))  # log(1 + exp(x))

plt.figure(figsize=(8, 5))
plt.plot(x, softplus, label="softplus(x)", linewidth=2)
plt.title("Softplus Activation Function")
plt.xlabel("x")
plt.ylabel("softplus(x)")
plt.grid(True)
plt.legend()
plt.show()

-> P7_453 is Right, all others are Left


In [ ]:
base_config_path = base_dir / "data/<person_ID>/configs/modular_initialization.yaml"
with open(base_config_path, "r") as f:
    base_config_template = yaml.safe_load(f)

base_config_template["parameters"]["activation_model"] = {
    "parameters": {
        "enforce_positive_weights": {"values": [False, True]},
        "model_type": {"value": "MatMul"},
        "n_freeze_epochs": {"value": 0},
        "init": {"values": ["default", "xavier_normal"]},
    }
}
base_config_template["parameters"]["wandb_project"]["value"] = "matmul_experiments"
base_config_template["parameters"]["learning_rate"]["values"] = [
    0.1,
    0.01,
    0.001,
    0.0001,
]
base_config_template["parameters"]["learning_rate"].pop("value", None)

base_config_template["parameters"]["dummy_repetition"]["value"] = 1
base_config_template["parameters"]["dummy_repetition"].pop("values", None)

for participant in participants:
    participant_config_path = (
        base_dir / f"data/{participant}/configs/modular_MatMul.yaml"
    )

    base_config = base_config_template.copy()
    base_config["name"] = participant

    if participant == "P7_453":
        base_config["parameters"]["targets"]["value"] = [
            ["Right", joint]
            for _, joint in base_config["parameters"]["targets"]["value"]
        ]
    else:
        base_config["parameters"]["targets"]["value"] = [
            ["Left", joint]
            for _, joint in base_config["parameters"]["targets"]["value"]
        ]
    with open(participant_config_path, "w") as f:
        yaml.dump(base_config, f)
    print(f"Copied config to {participant_config_path}")

In [ ]:
import subprocess

for participant in participants:
    hand = "Right" if participant == "P7_453" else "Left"
    print(f"Running training for {participant} with intact hand {hand}")

    # Run the command using subprocess
    subprocess.run(
        [
            "python",
            base_dir / "s4_train.py",
            "--person_dir",
            participant,
            "--intact_hand",
            hand,
            "--config_name",
            "modular_MatMul",
            "-hs",
        ],
        cwd=base_dir,
    )

In [ ]:
import copy

import wandb

api = wandb.Api()

# Project is specified by <entity/project-name>
runs = api.runs("MIT_biomech/matmul_experiments")

summary_list, config_list, name_list = [], [], []
for run in runs:
    # .summary contains the output keys/values for metrics like accuracy.
    #  We call ._json_dict to omit large files
    summary_list.append(run.summary._json_dict)

    # .config contains the hyperparameters.
    #  We remove special values that start with _.
    config_list.append({k: v for k, v in run.config.items() if not k.startswith("_")})

    # .name is the human-readable name of the run.
    name_list.append(run.name)

runs_df = pd.DataFrame(
    {"summary": summary_list, "config": config_list, "name": name_list}
)

runs_df_archive = copy.deepcopy(runs_df)

In [ ]:
runs_df = copy.deepcopy(runs_df_archive)

In [ ]:
runs_df["total_val_loss"] = runs_df["summary"].apply(
    lambda x: x.get("total_val_loss", None)
)
runs_df["total_test_loss"] = runs_df["summary"].apply(
    lambda x: x.get("total_test_loss", None)
)
runs_df["n_epochs"] = runs_df["summary"].apply(lambda x: x.get("_step", None))

# parameters: "learning_rate", "activation_model/init", "enforce_positive_weights"
runs_df["learning_rate"] = runs_df["config"].apply(
    lambda x: x.get("learning_rate", None)
)
runs_df["init"] = runs_df["config"].apply(
    lambda x: x.get("activation_model").get("init", None)
)
runs_df["enforce_positive_weights"] = runs_df["config"].apply(
    lambda x: x.get("activation_model").get("enforce_positive_weights", None)
)

In [ ]:
np.unique(runs_df["n_epochs"].values, return_counts=True)

In [ ]:
# set loss to infinity for failed runs
runs_df.loc[runs_df["n_epochs"] != 39, "total_val_loss"] = np.inf
runs_df.loc[runs_df["n_epochs"] != 39, "total_test_loss"] = np.inf

In [ ]:
runs_df.drop(columns=["summary", "config"], inplace=True)
runs_df.rename(columns={"name": "participant"}, inplace=True)

In [ ]:
runs_df["total_combined_loss"] = (
    runs_df["total_val_loss"] * 9 + runs_df["total_test_loss"] * 3
) / 12

In [ ]:
df_amputee = runs_df[runs_df["participant"].isin(amputees)].copy()
df_intact = runs_df[runs_df["participant"].isin(intact)].copy()

In [ ]:
# group by participant and init, then get the mean and std of total_val_loss
parameters = ["enforce_positive_weights", "init", "learning_rate"]
grouped_df = (
    runs_df.groupby(parameters)
    .agg(
        mean_val_loss=("total_val_loss", "mean"),
        std_val_loss=("total_val_loss", "std"),
        mean_test_loss=("total_test_loss", "mean"),
        std_test_loss=("total_test_loss", "std"),
        mean_combined_loss=("total_combined_loss", "mean"),
        std_combined_loss=("total_combined_loss", "std"),
    )
    .reset_index()
)
grouped_df_amputee = (
    df_amputee.groupby(parameters)
    .agg(
        mean_val_loss=("total_val_loss", "mean"),
        std_val_loss=("total_val_loss", "std"),
        mean_test_loss=("total_test_loss", "mean"),
        std_test_loss=("total_test_loss", "std"),
        mean_combined_loss=("total_combined_loss", "mean"),
        std_combined_loss=("total_combined_loss", "std"),
    )
    .reset_index()
)
grouped_df_intact = (
    df_intact.groupby(parameters)
    .agg(
        mean_val_loss=("total_val_loss", "mean"),
        std_val_loss=("total_val_loss", "std"),
        mean_test_loss=("total_test_loss", "mean"),
        std_test_loss=("total_test_loss", "std"),
        mean_combined_loss=("total_combined_loss", "mean"),
        std_combined_loss=("total_combined_loss", "std"),
    )
    .reset_index()
)

grouped_df.fillna(np.inf, inplace=True)
grouped_df_amputee.fillna(np.inf, inplace=True)
grouped_df_intact.fillna(np.inf, inplace=True)

In [ ]:
grouped_df

In [ ]:
grouped_df_amputee.loc[9, "mean_combined_loss"].round(3)